In [ ]:
!pip install ultralytics
!pip install easyocr
!pip install constants

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.7/872.7 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 59.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of constants to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
  Created wheel for constants: filename=constants-0.6.0-py3-none-any.whl size=5457 sha256=fb279e128c550347bf5cdd0ddef982647a74a0f87f7da4ee41215a4cb01d7dcf
  Stored in directory: /root/.cache/pip/wheels/5b/96/3c/386c2342a8a1bdd317f2f250bd076c13938c6f598c4a40ec14
Successfully built constants


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import easyocr
import pandas as pd
import cv2
import numpy as np
import requests
import re
import constants
import os
import requests
import pandas as pd
import multiprocessing
import time
from time import time as timer
from tqdm import tqdm
import numpy as np
from pathlib import Path
from functools import partial
import requests
import urllib
from PIL import Image

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os

# Change directory to where your dataset and YAML file are located
# Add amazon_hackathon folder shortcut to MyDrive
os.chdir('/content/drive/MyDrive/amazon_ml_challenge')

Mounted at /content/drive


In [ ]:
model = YOLO("/content/drive/MyDrive/amazon_ml_challenge/best.pt")

In [ ]:
## Functions

def f(asked_entity, cls, xyxy, conf):
    ans = []
    # Create a list of tuples containing bounding box and confidence value
    for idx, i in enumerate(cls):
        if i == asked_entity and conf[idx] > 0.1:
            ans.append((xyxy[idx].tolist(), conf[idx]))

    # Sort the list by the confidence value in descending order
    ans.sort(key=lambda x: x[1], reverse=True)

    sorted_xyxy = [item[0] for item in ans]
    return sorted_xyxy

def crop_image_cv2(image, coords):
    # Extract coordinates
    x_min, y_min, x_max, y_max = coords

    # Crop the image using slicing
    cropped_img = image[int(y_min):int(y_max), int(x_min):int(x_max)]

    return cropped_img

# Download Image Code
def common_mistake(unit):
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, constants.allowed_units))
    return number, unit


def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)

In [ ]:
## Functions

def f(asked_entity, cls, xyxy, conf):
    ans = []
    # Create a list of tuples containing bounding box and confidence value
    for idx, i in enumerate(cls):
        if i == asked_entity and conf[idx] > 0.1:
            ans.append((xyxy[idx].tolist(), conf[idx]))

    # Sort the list by the confidence value in descending order
    ans.sort(key=lambda x: x[1], reverse=True)

    sorted_xyxy = [item[0] for item in ans]
    return sorted_xyxy

def crop_image_cv2(image, coords):
    # Extract coordinates
    x_min, y_min, x_max, y_max = coords

    # Crop the image using slicing
    cropped_img = image[int(y_min):int(y_max), int(x_min):int(x_max)]

    return cropped_img

# Download Image Code
def common_mistake(unit):
    if unit in constants.allowed_units:
        return unit
    if unit.replace('ter', 'tre') in constants.allowed_units:
        return unit.replace('ter', 'tre')
    if unit.replace('feet', 'foot') in constants.allowed_units:
        return unit.replace('feet', 'foot')
    return unit

def parse_string(s):
    s_stripped = "" if s==None or str(s)=='nan' else s.strip()
    if s_stripped == "":
        return None, None
    pattern = re.compile(r'^-?\d+(\.\d+)?\s+[a-zA-Z\s]+$')
    if not pattern.match(s_stripped):
        raise ValueError("Invalid format in {}".format(s))
    parts = s_stripped.split(maxsplit=1)
    number = float(parts[0])
    unit = common_mistake(parts[1])
    if unit not in constants.allowed_units:
        raise ValueError("Invalid unit [{}] found in {}. Allowed units: {}".format(
            unit, s, constants.allowed_units))
    return number, unit


def create_placeholder_image(image_save_path):
    try:
        placeholder_image = Image.new('RGB', (100, 100), color='black')
        placeholder_image.save(image_save_path)
    except Exception as e:
        return

def download_image(image_link, save_folder, retries=3, delay=3):
    if not isinstance(image_link, str):
        return

    filename = Path(image_link).name
    image_save_path = os.path.join(save_folder, filename)

    if os.path.exists(image_save_path):
        return

    for _ in range(retries):
        try:
            urllib.request.urlretrieve(image_link, image_save_path)
            return
        except:
            time.sleep(delay)

    create_placeholder_image(image_save_path) #Create a black placeholder image for invalid links/images

def download_images(image_links, download_folder, allow_multiprocessing=True):
    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    if allow_multiprocessing:
        download_image_partial = partial(
            download_image, save_folder=download_folder, retries=3, delay=3)

        with multiprocessing.Pool(64) as pool:
            list(tqdm(pool.imap(download_image_partial, image_links), total=len(image_links)))
            pool.close()
            pool.join()
    else:
        for image_link in tqdm(image_links, total=len(image_links)):
            download_image(image_link, save_folder=download_folder, retries=3, delay=3)

In [ ]:
entity_number_map = {
    "width": 0,
    "depth": 1,
    "height": 2,
    "item_weight": 3,
    "maximum_weight_recommendation": 4,
    "voltage": 5,
    "wattage": 6,
    "item_volume": 7
}

number_entity_map = {value: key for key, value in entity_number_map.items()}

entity_unit_map = {
  "width": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "depth": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "height": {
    "centimetre",
    "foot",
    "millimetre",
    "metre",
    "inch",
    "yard"
  },
  "item_weight": {
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  },
  "maximum_weight_recommendation": {
    "milligram",
    "kilogram",
    "microgram",
    "gram",
    "ounce",
    "ton",
    "pound"
  },
  "voltage": {
    "millivolt",
    "kilovolt",
    "volt"
  },
  "wattage": {
    "kilowatt",
    "watt"
  },
  "item_volume": {
    "cubic foot",
    "microlitre",
    "cup",
    "fluid ounce",
    "centilitre",
    "imperial gallon",
    "pint",
    "decilitre",
    "litre",
    "millilitre",
    "quart",
    "cubic inch",
    "gallon"
  }
}

entity_unit_variations_map = {
    "width": {
        "centimetre": ["cm", "centimeter", "centimetre","c", "ce", "cen", "cent", "centi"],
        "foot": ["ft", "foot", "'", "f", "fo"],
        "millimetre": ["mm", "millimeter", "millimetre", "mi", "milli"],
        "metre": ["m", "meter", "metre"],
        "inch": ["in", '"', "inch", "i"],
        "yard": ["yd", "yard", "y"]
    },
    "depth": {
        "centimetre": ["cm", "centimeter", "centimetre","c", "ce", "cen", "cent", "centi"],
        "foot": ["ft", "foot", "'", "f", "fo"],
        "millimetre": ["mm", "millimeter", "millimetre", "mi", "milli"],
        "metre": ["m", "meter", "metre"],
        "inch": ["in", '"', "inch", "i"],
        "yard": ["yd", "yard", "y"]
    },
    "height": {
        "centimetre": ["cm", "centimeter", "centimetre","c", "ce", "cen", "cent", "centi"],
        "foot": ["ft", "foot", "'", "f", "fo"],
        "millimetre": ["mm", "millimeter", "millimetre", "mi", "milli"],
        "metre": ["m", "meter", "metre"],
        "inch": ["in", '"', "inch", "i"],
        "yard": ["yd", "yard", "y"]
    },
    "item_weight": {
        "milligram": ["mg", "milligram", "m"],
        "kilogram": ["kg", "kilogram", "k"],
        "gram": ["g", "gram"],
        "ounce": ["oz", "ounce", "o"],
        "pound": ["lb", "pound", "p", "l"]
    },
    "maximum_weight_recommendation": {
        "milligram": ["mg", "milligram", "m"],
        "kilogram": ["kg", "kilogram", "k"],
        "gram": ["g", "gram"],
        "ounce": ["oz", "ounce", "o"],
        "pound": ["lb", "pound", "p", "l"]
    },
    "voltage": {
        "volt": ["v", "volt"],
        "kilovolt": ["kv", "kilovolt", "k"],
        "millivolt": ["mv", "millivolt", "m"]
    },
    "wattage": {
        "kilowatt": ["kw", "KW", "kilowatt", "KiloWatt", "k"],
        "watt": ["W", "w", "watt", "Watt", "wa", "wat"]
    },
    "item_volume": {
        "litre": ["l", "litre", "liter"],
        "millilitre": ["ml", "millilitre", "milliliter", "m"],
        "gallon": ["gal", "gallon", "g", "ga", "gallo"],
        "cubic foot": ["cu ft", "cubic foot", "cf", "c f", "c.f."],
        "cup": ["cup", "cu"],
        "fluid ounce": ["fl oz", "fluid ounce", "f", "fl", "fl o", "flu", "flui", "fluid", "fluid o", "fluid on", "fluid oun", "fluid ounc"]
    }
}



detect_unit_map = {}

for entity, units in entity_unit_variations_map.items():
    detect_unit_map[entity] = {}
    for standard_unit, variations in units.items():
        for variation in variations:
            detect_unit_map[entity][variation] = standard_unit


#Useful Part:

In [ ]:
def image_to_entity_value(asked_entity, img):
    reader_out = reader.readtext(img, detail=0)
    if isinstance(reader_out, list) and len(reader_out) > 0:
        temp_entity_value = reader_out[0]
    else:
        temp_entity_value = reader_out

    entity_value = ""
    flag=0
    for i in temp_entity_value:
        if not i.isalpha():
            if i in " 012345789.":
                entity_value += i
        else:
            for entity, entity_real in detect_unit_map[asked_entity].items():
                if entity in temp_entity_value.lower():
                  if len(entity_value) > 0 and not entity_value[-1].isspace():
                    entity_value += ' '
                  entity_value += entity_real
                  flag=1
                  break
            if flag:
                break
    return entity_value

In [ ]:
def run_master_pipeline(asked_entity, image_path):
    # step - 1 YOLO Result
    result = model(image_path)
    width = list(result[0].boxes.orig_shape)[0]
    height = list(result[0].boxes.orig_shape)[1]
    cls = result[0].boxes.cls.tolist()
    conf = result[0].boxes.conf.tolist()
    xyxy = list(result[0].boxes.xyxy)

    result[0].show()

    x = f(entity_number_map[asked_entity], cls, xyxy, conf)

    if len(x) == 0:
        print("x is zero")
        return [""]

    try:
      img = cv2.imread(image_path)
    except:
      return [""]

    temp_entity_value_arr = []
    for i in x:
        cropped = crop_image_cv2(img, i)

        # if image is sideways, rotate it
        height, width, channels = img.shape
        if height > width:
            cropped = cv2.rotate(cropped, cv2.ROTATE_90_CLOCKWISE)

        temp_entity_value_arr.append(image_to_entity_value(asked_entity, cropped))
        # plt.imshow(cropped)

    return temp_entity_value_arr

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/amazon_ml_challenge/best.pt'
DOWNLOADS_FOLDER_TEST = '/content/drive/MyDrive/amazon_ml_challenge/download_test'
OUTPUT_PATH = '/content/drive/MyDrive/amazon_ml_challenge/output_folder'

### CHANGE THE NUMBERS (reference on whiteboard)
OUTPUT_FILE_NUMBER = 6
START_INDEX = 50000
END_INDEX = 60000

OUTPUT_FILE = f"test_output_{OUTPUT_FILE_NUMBER}.csv"

In [ ]:
if not os.path.exists(DOWNLOADS_FOLDER_TEST):
    os.makedirs(DOWNLOADS_FOLDER_TEST)
if not os.path.exists(OUTPUT_PATH):
    os.makedirs(OUTPUT_PATH)

In [ ]:
global model
global reader
model = YOLO(MODEL_PATH)
reader = easyocr.Reader(['en'])

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/amazon_ml_challenge/test.csv')
df.head()

,index,image_link,group_id,entity_name
0,0,https://m.media-amazon.com/images/I/110EibNycl...,156839,height
1,1,https://m.media-amazon.com/images/I/11TU2clswz...,792578,width
2,2,https://m.media-amazon.com/images/I/11TU2clswz...,792578,height
3,3,https://m.media-amazon.com/images/I/11TU2clswz...,792578,depth
4,4,https://m.media-amazon.com/images/I/11gHj8dhhr...,792578,depth


In [ ]:
# Download all images

# image_links_test = []

# for row_index, row in df.iterrows():
#     image_link = row.image_link
#     image_links_test.append(image_link)

# download_images(image_links_test, DOWNLOADS_FOLDER_TEST)

100%|██████████| 131187/131187 [05:45<00:00, 380.15it/s] 


In [ ]:
# Check number of downloaded images
# num_files = len(os.listdir("/content/drive/MyDrive/amazon_ml_challenge/download_test"))
# print(num_files)

90666


In [ ]:
# Slice dataset according to start index and end index
temp_df = df.iloc[START_INDEX:END_INDEX]

In [ ]:
## Generate test outputs

test_predictions = []
for row_index, row in temp_df.iterrows():
    image_link = row.image_link
    entity_name = row.entity_name
    index = row['index']

    image_path = os.path.join(DOWNLOADS_FOLDER_TEST, image_link.split("/")[-1])

    try:
      out = run_master_pipeline(entity_name, image_path)
      test_predictions.append((index, out[0]))
      print((index, out[0]))
    except:
      test_predictions.append((index, ""))
      print((index, "errorr"))


output_df = pd.DataFrame(test_predictions, columns=['index', 'entitiy_name'])
output_df.to_csv(OUTPUT_FILE, index=False)